In [45]:
import requests
import polars as pl
import numpy as np
import json

In [ ]:
pl.Config(tbl_rows=10)

test_data = pl.read_csv('../data/User0_credit_card_transactions.csv')
test_data.head()

In [ ]:
def preprocess_dataset(data_df):
    """ Preprocess dataset for service. """
    new_data_df = data_df.with_columns(
        Hour = pl.col("Time").map_elements(
            lambda x: x[:2]).cast(pl.Int64, strict=True),
        Minute = pl.col("Time").map_elements(
            lambda x: x[3:]).cast(pl.Int64, strict=True),
    ).drop("Time").with_columns(
        pl.col("Amount").map_elements(
            lambda x: x.replace("$", "")).cast(pl.Float64, strict=True)
    ).with_columns(
        pl.col("Merchant State").fill_null("ONLINE")
    ).with_columns(
        pl.col("Zip").cast(pl.String, strict=True).fill_null("ONLINE")
    ).with_columns(
        pl.col("Errors?").fill_null(value="No")
    ).with_columns(
        pl.col("Is Fraud?").map_elements(
            lambda x: 0 if x == "No" else 1
        )
    )
    return new_data_df
test_data = preprocess_dataset(test_data)
test_data

In [ ]:
json_data = test_data.to_pandas().to_json(orient='records')


In [51]:
response = requests.post(
    'http://0.0.0.0:3000/predict',
    data=json_data
)

response

<Response [200]>

In [50]:
predictions = response.json()
result = pl.concat(
    [test_data, pl.DataFrame({"Predicted_Is_Fraud?":predictions})],
    how="horizontal")
result

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Hour,Minute,Predicted_Is_Fraud?
i64,i64,i64,i64,i64,f64,str,i64,str,str,str,i64,str,i64,i64,i64,i64
0,0,2002,9,1,134.09,"""Swipe Transact…",3527213246127876953,"""La Verne""","""CA""","""91750.0""",5300,"""No""",0,6,21,0
0,0,2002,9,1,38.48,"""Swipe Transact…",-727612092139916043,"""Monterey Park""","""CA""","""91754.0""",5411,"""No""",0,6,42,0
0,0,2002,9,2,120.34,"""Swipe Transact…",-727612092139916043,"""Monterey Park""","""CA""","""91754.0""",5411,"""No""",0,6,22,0
0,0,2002,9,2,128.95,"""Swipe Transact…",3414527459579106770,"""Monterey Park""","""CA""","""91754.0""",5651,"""No""",0,17,45,0
0,0,2002,9,3,104.71,"""Swipe Transact…",5817218446178736267,"""La Verne""","""CA""","""91750.0""",5912,"""No""",0,6,23,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,4,2009,2,12,9.71,"""Swipe Transact…",4060646732831064559,"""La Verne""","""CA""","""91750.0""",5411,"""Technical Glit…",0,13,3,0
0,4,2009,2,22,27.94,"""Swipe Transact…",-8129520121141002186,"""La Verne""","""CA""","""91750.0""",5211,"""No""",0,11,58,0
0,4,2009,2,24,13.97,"""Swipe Transact…",5474320255037684877,"""Hawthorne""","""CA""","""90250.0""",5912,"""No""",0,16,53,0
